In [1]:
import os
import unittest
import requests
import urllib3
from utils_demo import *
from loguru import logger
from unittest.mock import patch
from pprint import pprint
from os.path import dirname, abspath
import warnings
warnings.filterwarnings('ignore')
from performance_data.performance_data import PhaseData, DataTable
from performance_data import DATASET_FILE
from app_decomposer import DEFAULT_CONFIGURATION, KIWI_CONFIG, CURRENT_DIR, API_DICT_TS, IOI_SAMPLING_PERIOD, DATASET_SOURCE
from app_decomposer.api_connector import request_delegator
from app_decomposer.config_parser import Configuration
from app_decomposer.api_connector import TimeSeries
print(DATASET_SOURCE)
print(DATASET_FILE)

/home_nfs/mimounis/iosea-wp3-recommandation-system/performance_data/performance_data/dataset/performance_model_dataset.csv
/home_nfs/mimounis/iosea-wp3-recommandation-system/performance_data/performance_data/dataset/performance_model_dataset_complete.csv


In [10]:
# checking dataset values
print_readable_values("/home_nfs/mimounis/iosea-wp3-recommandation-system/performance_data/performance_data/dataset/performance_model_dataset_complete.csv", ["volume", "IOsize", "ioi_bw", "lfs_bw","sbb_bw"])

    job_id     volume   mode IOpattern    IOsize  nodes     ioi_bw     lfs_bw  \
0     6873    1.64 KB   read      uncl   328.4 B      1    164.2 B   491.4 MB   
1     6873    9.62 GB   read      rand   1.05 MB      1    1.92 GB   596.3 MB   
2     6873   48.12 GB   read      rand   5.24 MB      1    4.81 GB  634.12 MB   
3     6873   96.24 GB   read      rand  10.48 MB      1    6.42 GB  730.83 MB   
4     6873   10.49 GB  write      rand   1.05 MB      1  524.29 MB  214.55 MB   
5     6873   52.43 GB  write      rand   5.24 MB      1  582.54 MB  260.76 MB   
6     6873  104.86 GB  write      rand  10.49 MB      1   676.5 MB  265.19 MB   
7     6873         0B   read      uncl        0B      1         0B         0B   
8     6873    1.64 KB   read      uncl   328.4 B      1    164.2 B  492.13 MB   
9     6873    9.62 GB   read      rand   1.05 MB      1    1.92 GB  593.82 MB   
10    6873   48.12 GB   read      rand   5.24 MB      1    4.81 GB   640.2 MB   
11    6873   96.24 GB   read

In [9]:
# checking dataset values
print_readable_values("/home_nfs/mimounis/iosea-wp3-recommandation-system/performance_data/performance_data/dataset/performance_model_dataset_complete_2.csv", ["volume", "IOsize", "ioi_bw", "lfs_bw","sbb_bw"])

    job_id     volume   mode IOpattern    IOsize  nodes     ioi_bw     lfs_bw  \
0     6873    1.64 KB   read      uncl   328.4 B      1    164.2 B     4.0 KB   
1     6873    9.62 GB   read      rand   1.05 MB      1    1.92 GB  658.49 MB   
2     6873   48.12 GB   read      rand   5.24 MB      1    4.81 GB     3.1 GB   
3     6873   96.24 GB   read      rand  10.48 MB      1    6.42 GB    7.15 GB   
4     6873   10.49 GB  write      rand   1.05 MB      1  524.29 MB  230.18 MB   
5     6873   52.43 GB  write      rand   5.24 MB      1  582.54 MB  322.43 MB   
6     6873  104.86 GB  write      rand  10.49 MB      1   676.5 MB  330.36 MB   
7     6873         0B   read      uncl        0B      1         0B         0B   
8     6873    1.64 KB   read      uncl   328.4 B      1    164.2 B     8.6 KB   
9     6873    9.62 GB   read      rand   1.05 MB      1    1.92 GB  664.72 MB   
10    6873   48.12 GB   read      rand   5.24 MB      1    4.81 GB    3.14 GB   
11    6873   96.24 GB   read

In [ ]:
%%capture
# Connect directly to kiwi0
logger.remove()
ioi_config = Configuration(path=KIWI_CONFIG)
job_id = 6873 #5168
cd = ComplexDecomposer(job_id=job_id, v0_threshold=1e-8, config=ioi_config)
# Launch decomposition on the signal
representation = cd.get_job_representation(merge_clusters=True)
pprint(representation)
# This is the app encoding representation for Execution Simulator
pprint(f"compute={representation['events']}, reads={representation['read_volumes']}" 
      f"read_bw={representation['read_bw']}, writes={representation['write_volumes']}"
      f"write_bw={representation['write_bw']}, read_pattern={representation['read_pattern']}")
# Normalize signals to seconds and MB
timestamps = (cd.timestamps - cd.timestamps[0])/5
original_read = cd.read_signal/1e6
original_write = cd.write_signal/1e6

fig = plot_detected_phases(jobid=job_id, merge=True, show_phases=True, 
                           ts=(timestamps, original_read, original_write),
                           width=800, height=600)
fig.show() 


In [ ]:
# Extract phases table and features
phases_features = cd.get_phases_features(representation,
                                         job_id = job_id,
                                         update_csv=True)

# display the csv table
# DATASET_SOURCE
print(pd.read_csv(DATASET_SOURCE, index_col=False))

In [ ]:
# Feed three columns of the CSV table by launching tiers perf measurements
# with nfw_bw | lfs_bw | sbb_bw
print(DATASET_FILE)
#targets = dict(lfs="/fsiof/mimounis/tmp", nfs="/scratch/mimounis/tmp")
targets = dict(lfs="/fsiof/mimounis/tmp")
dt = DataTable(targets, accelerator=True)
df = dt.get_performance_table()

In [ ]:
# display the csv table
# DATASET_SOURCE
print(pd.read_csv(DATASET_FILE, index_col=False))

In [ ]:
# Apply same BW as measured by IOI (from representation)
read_bw = list(map(lambda x: x/1e6, representation['read_bw']))
write_bw = list(map(lambda x: x/1e6, representation['write_bw']))
ioi_bw = list(map(lambda x, y: (x + y), read_bw, write_bw))
print(ioi_bw)


In [ ]:
# Get the BW predicted if the app run on some tiers // no model, just indexed tabular
df = pd.read_csv(DATASET_FILE, index_col=False)
df = pd.read_csv("/home_nfs/mimounis/iosea-wp3-recommandation-system/performance_data/performance_data/dataset/performance_model_dataset_complete_2.csv", index_col=False)
job_df = df[df["job_id"] == job_id]
perf = {}
for tier_bw in ["lfs_bw", "sbb_bw"]:
    rough_bw = list(job_df[[tier_bw]].to_numpy().flatten())
    # perf adjusted to IOI
    perf[tier_bw] = list(map(lambda x: x*5/1e6, rough_bw))       
perf["ioi_bw"] = list(map(lambda x: x*5/1e6, job_df[["ioi_bw"]].to_numpy().flatten()))
print(perf)


In [ ]:
# Uncorrelate reads and writes to avoid simulator crash (not accepting mixed R/W)
events = representation['events']
reads = list(map(lambda x, y: x if x > y else 0, representation['read_volumes'], representation["write_volumes"]))
writes = list(map(lambda x, y: x if x > y else 0, representation['write_volumes'], representation["read_volumes"]))
print(events)
print(reads)
print(writes)


In [ ]:
# prepare simulated on SBB
sim_time, sim_read_bw, sim_write_bw = simulate_app(events, reads, writes,
                                                   perf["sbb_bw"], app_name="job#6873")
print(sim_time[-1])

In [ ]:
# get the ground truth = experimental job of the same app running the the specified tier for verification.
cd = ComplexDecomposer(job_id=6871, v0_threshold=1e-8, config=ioi_config)
timestamps = (cd.timestamps - cd.timestamps[0])/5000
original_read = cd.read_signal/1e6
original_write = cd.write_signal/1e6
# 5168-5175: nfs
# 5182-5184: fs1
# 5192-5194: sbb
fig = display_original_sim_signals((sim_time, sim_read_bw, sim_write_bw),
                                   (timestamps, original_read, original_write),
                                   str_org="Experimental lfs sbb",
                                   str_sim="Predicted",
                                   width=800, height=600)
fig.show()
# timestamps, experimental_read, experimental_write = get_job_timeseries_from_file_as_array(job_id=5168)
# timestamps = (timestamps - timestamps[0])/5
# experimental_read = experimental_read/1e6
# experimental_write = experimental_write/1e6

In [ ]:
print(sim_time)

In [ ]:
sim_time, sim_read_bw, sim_write_bw = simulate_app(representation['events'],
                                                   representation['read_volumes'],
                                                   representation['write_volumes'], 
                                                   perf["ioi_bw"], app_name="job#6873")


fig = display_original_sim_signals((sim_time, sim_read_bw, sim_write_bw),
                                   (timestamps, experimental_read, experimental_write),
                                   str_org="Experimental (ioi_bw)",
                                   str_sim="Predicted",
                                   width=1000, height=900)
fig.show()

In [ ]:
print("coco")